In [18]:
import json
import pandas as pd

with open('data.json', 'r') as f:
    data = json.load(f)

In [19]:
with open('data.csv', 'w') as f:
    f.write("username,comment,didDonate,currency,amount\n")
    for item in data:        
        comment_content=item['comment'].replace(','," ").replace("\n"," ").replace("\r"," ")
        if item['super_thanks_amount']=='':
            donated=False
            amount=0
            currency=None
        else:
            donated=True
            amount=item['super_thanks_amount'][1:].replace(',','')
            currency=item['super_thanks_amount'][0]
        f.write("{},{},{},{},{}\n".format(item['author'],comment_content,donated,currency,amount))        

In [27]:
df=pd.read_csv('data.csv')
set(df['currency'].dropna().tolist())

{'$', '₹'}

In [36]:
donation_stats={
    "amount_usd":0,
    "amount_usd_qty":0,
    "amount_inr":0,    
    "amount_inr_qty":0
}
donation_stats['amount_usd']=sum(df[df['currency']=="$"]['amount'].tolist())
donation_stats['amount_usd_qty']=len(df[df['currency']=="$"]['amount'].tolist())
donation_stats['amount_inr']=sum(df[df['currency']=="₹"]['amount'].tolist())
donation_stats['amount_inr_qty']=len(df[df['currency']=="₹"]['amount'].tolist())
donation_stats

{'amount_usd': 109.99,
 'amount_usd_qty': 3,
 'amount_inr': 21576.0,
 'amount_inr_qty': 22}

In [43]:
total_inr=donation_stats['amount_usd']*85.39+donation_stats['amount_inr']
total_usd=donation_stats['amount_usd']+donation_stats['amount_inr']/85.39
print(f"₹{total_inr:.2f},${total_usd:.2f}")

₹30968.05,$362.67


In [54]:
from textblob import TextBlob

sentiment_scores=[]
for index,comment in enumerate(df['comment']):    
    blob=TextBlob(str(comment))
    sentiment_score=blob.sentiment.polarity    
    sentiment_scores.append(sentiment_score)
sentiment_scores=pd.Series(sentiment_scores)
sentiment_scores.describe()

count    1098.000000
mean        0.132378
std         0.326732
min        -1.000000
25%         0.000000
50%         0.056004
75%         0.308949
max         1.000000
dtype: float64